Tutorial from https://chanzuckerberg.github.io/cellxgene-census/notebooks/experimental/pytorch.html

In [28]:
import cellxgene_census
import cellxgene_census.experimental.ml as census_ml
import tiledbsoma as soma

import torch
import torch.nn as nn
import torch.nn.functional as F

import pandas as pd

import os
import pickle



In [2]:
cellxgene_census.__version__

'1.11.1'

## Load the Saved Outputs from McCell_preprocessing

# Are any of these dependent on the order of the X data in the preprocessing. If so, we would want to be careful

- cell_parent_mask
- Mapping_dict
- Ontology_df
- Internal_values
- leaf_values


In [20]:
# define the date in yyyy-mm-dd format
date = '2024-02-22'

In [23]:
ontology_df_name = date + '_ontology_df.csv'
ontology_df = pd.read_csv(ontology_df_name,index_col=0)


mapping_dict_name = date + '_mapping_dict_df.csv'
mapping_dict_df = pd.read_csv(mapping_dict_name,index_col=0)
mapping_dict = mapping_dict_df.T.to_dict('list')
# the values are stored as a list. convert to single value
for key, value in mapping_dict.items():
    mapping_dict[key] = value[0]

leaf_values_name = date + '_leaf_values'
internal_values_name = date + '_internal_values'
with open(leaf_values_name,'rb') as fp:
    leaf_values = pickle.load(fp)
with open(internal_values_name,'rb') as fp:
    internal_values = pickle.load(fp)


cell_parent_mask_name = date + '_cell_parent_mask.pt'
cell_parent_mask = torch.load(cell_parent_mask_name)

FileNotFoundError: [Errno 2] No such file or directory: '2024-02-22_ontology_df.csv'

## Build the Experiment and the DataPipe

In [22]:
# gene and cell type info stored on Turbo
os.chdir('/nfs/turbo/umms-welchjd/mccell')

First, let's load the gene list and cell type list that we want from the Census. Then we construct the ```var_val_filter``` and ```obs_val_filter``` for querying the census.

In [10]:
# load gene list
biomart = pd.read_csv('mart_export.txt')

coding_only = biomart[biomart['Gene type'] == 'protein_coding']

gene_list = coding_only['Gene stable ID'].to_list()

var_val_filter = '''feature_id in {}'''.format(gene_list)

# load the cell type list
cell_type_list_name = 'cell_type_list.txt'
with open(cell_type_list_name,'rb') as fp:
    cell_type_list = pickle.load(fp)

obs_val_filter = '''assay == "10x 3\' v3" and is_primary_data == True and cell_type_ontology_term_id in {}'''.format(cell_type_list)


In [55]:
#organism = "Homo sapiens"
col_names = {"obs": ["cell_type_ontology_term_id"]}


In [6]:
# open the Census and create the Experiment
census = cellxgene_census.open_soma(uri = "/scratch/welchjd_root/welchjd99/fujoshua/soma")
experiment = census["census_data"]["homo_sapiens"]


In [12]:
experiment_datapipe = census_ml.ExperimentDataPipe(
    experiment,
    measurement_name="RNA",
    X_name="raw",
    obs_query=soma.AxisQuery(value_filter=obs_val_filter),
    var_query=soma.AxisQuery(value_filter=var_val_filter),
    obs_column_names=["cell_type_ontology_term_id"],
    batch_size=128,
    shuffle=True,
    soma_chunk_size=10_000,
)


In [26]:
experiment_datapipe.shape[1]

19966

In [17]:
train_datapipe, test_datapipe = experiment_datapipe.random_split(weights={"train": 0.8, "test": 0.2}, seed=1)


In [19]:
train_dataloader = census_ml.experiment_dataloader(train_datapipe)
test_dataloader = census_ml.experiment_dataloader(test_datapipe)

## Build Neural Network Classifier

First, we need to select and define the input and output dimensions from the data. The number of neurons for the hidden nodes is defined manually.

In [ ]:
# number of features (len of X cols)
# select the number of gene columns
input_dim = experiment_datapipe.shape[1]#X_train.size(dim=1) #adata.X.shape[1] 

# number of neurons for hidden layers
hidden_layer_1 = 256
hidden_layer_2 = 128

# number of classes (unique of y)
output_dim = torch.unique(y_train[y_train >= 0]).size(dim=0) #labels['encoded_labels'].nunique()

print(input_dim,hidden_layer_1,hidden_layer_2,output_dim)

In [29]:
class Network(nn.Module):
    def __init__(self):
        super(Network, self).__init__()
        self.linear1 = nn.Linear(input_dim,hidden_layer_1)
        self.linear2 = nn.Linear(hidden_layer_1,hidden_layer_2)
        self.linear3 = nn.Linear(hidden_layer_2,output_dim)
        self.bn1 = nn.BatchNorm1d(hidden_layer_1)
        self.bn2 = nn.BatchNorm1d(hidden_layer_2)
        
    def forward(self,x):
        x = self.linear1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.linear3(x)
        x = F.softmax(x,dim=1)
        return x
    
    def get_last_layer(self,x):
        x = self.linear1(x)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.linear2(x)
        x = self.bn2(x)
        x = F.relu(x)
        x = self.linear3(x)
        return x

In [9]:
experiment = census["census_data"]["homo_sapiens"]


In [10]:
experiment_datapipe = census_ml.ExperimentDataPipe(
    experiment,
    measurement_name="RNA",
    X_name="raw",
    obs_query=soma.AxisQuery(value_filter="tissue_general == 'tongue' and is_primary_data == True"),
    obs_column_names=["cell_type"],
    batch_size=128,
    shuffle=True,
    soma_chunk_size=10_000,
)


In [11]:
experiment_datapipe.shape

(15020, 60664)

In [12]:
train_datapipe, test_datapipe = experiment_datapipe.random_split(weights={"train": 0.8, "test": 0.2}, seed=1)


In [13]:
experiment_dataloader = census_ml.experiment_dataloader(train_datapipe)


In [15]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()  # noqa: UP008
        self.linear = torch.nn.Linear(input_dim, output_dim)

    def forward(self, x):
        outputs = torch.sigmoid(self.linear(x))
        return outputs


In [16]:
def train_epoch(model, train_dataloader, loss_fn, optimizer, device):
    model.train()
    train_loss = 0
    train_correct = 0
    train_total = 0

    for batch in train_dataloader:
        optimizer.zero_grad()
        X_batch, y_batch = batch

        X_batch = X_batch.float().to(device)

        # Perform prediction
        outputs = model(X_batch)

        # Determine the predicted label
        probabilities = torch.nn.functional.softmax(outputs, 1)
        predictions = torch.argmax(probabilities, axis=1)

        # Compute the loss and perform back propagation

        # Exclude the cell_type labels, which are in the second column
        y_batch = y_batch[:, 1]
        y_batch = y_batch.to(device)

        train_correct += (predictions == y_batch).sum().item()
        train_total += len(predictions)

        loss = loss_fn(outputs, y_batch.long())
        train_loss += loss.item()
        loss.backward()
        optimizer.step()

    train_loss /= train_total
    train_accuracy = train_correct / train_total
    return train_loss, train_accuracy


In [18]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

# The size of the input dimension is the number of genes
input_dim = experiment_datapipe.shape[1]

# The size of the output dimension is the number of distinct cell_type values
cell_type_encoder = experiment_datapipe.obs_encoders["cell_type"]
output_dim = len(cell_type_encoder.classes_)

model = LogisticRegression(input_dim, output_dim).to(device)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-05)

for epoch in range(3):
    train_loss, train_accuracy = train_epoch(model, experiment_dataloader, loss_fn, optimizer, device)
    print(f"Epoch {epoch + 1}: Train Loss: {train_loss:.7f} Accuracy {train_accuracy:.4f}")


Epoch 1: Train Loss: 0.0185056 Accuracy 0.2145
Epoch 2: Train Loss: 0.0162072 Accuracy 0.2665
Epoch 3: Train Loss: 0.0149051 Accuracy 0.3136


In [19]:
experiment_dataloader = census_ml.experiment_dataloader(test_datapipe)
X_batch, y_batch = next(iter(experiment_dataloader))


In [20]:
model.eval()

model.to(device)
outputs = model(X_batch.to(device))

probabilities = torch.nn.functional.softmax(outputs, 1)
predictions = torch.argmax(probabilities, axis=1)

display(predictions)


tensor([1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1,
        7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 7, 1, 1, 1, 1, 1, 7,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 7, 1, 1, 7, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 7, 7, 1, 1, 1, 1, 1])

In [3]:
census = cellxgene_census.open_soma(uri = "/scratch/welchjd_root/welchjd99/fujoshua/soma")

In [4]:
with cellxgene_census.open_soma(uri = "/scratch/welchjd_root/welchjd99/fujoshua/soma") as census:
    cell_metadata = census["census_data"]["homo_sapiens"].obs.read(
        value_filter = "sex == 'female' and cell_type in ['microglial cell', 'neuron']",
        column_names = ["assay", "cell_type", "tissue", "tissue_general", "suspension_type", "disease"]
    )


In [7]:
cell_metadata

In [ ]:
census

In [5]:
census["census_data"]["homo_sapiens"]

<Experiment 'file:///scratch/welchjd_root/welchjd99/fujoshua/soma/census_data/homo_sapiens' (CLOSED for 'r')>

In [6]:
census["census_data"]["homo_sapiens"].obs.read(
        value_filter = "sex == 'female' and cell_type in ['microglial cell', 'neuron']",
        column_names = ["assay", "cell_type", "tissue", "tissue_general", "suspension_type", "disease"]
    )

TileDBError: [TileDB::Array] Error: Cannot get array schema; Array is not open